<a href="https://colab.research.google.com/github/dhahbimohamed/PC-Parts-Recommender-System/blob/main/Notebooks/01_Scrape_MohamedDhahbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scrape Data**

# **Install what i need**

In [3]:
!pip install requests beautifulsoup4 pandas lxml fake_useragent


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.0 MB/s eta 0:00:00


In [4]:
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.12/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at async Registry._validateHostRequirements (/usr/local/lib/python3.12/dist

# **CPUs**

In [5]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.asyncio import tqdm
import time

# SPEC EXTRACTOR
def extract_specs(text):
    text = text.replace(",", ".").replace("-", " ")
    specs = {}

    # Cores
    match = re.search(r'(\d+)\s*(?:core|coeur|coeurs|cores?|C)\b', text, re.I)
    specs["Cores"] = match.group(1) if match else None

    # Threads
    match = re.search(r'(\d+)\s*(?:thread|threads|T)\b', text, re.I)
    specs["Threads"] = match.group(1) if match else None

    # Frequency (GHz)
    match = re.search(r'(\d+(?:\.\d+)?)\s*(?:GHz|GHZ)', text, re.I)
    specs["Frequency (GHz)"] = match.group(1) if match else None

    # Cache (MB)
    match = re.search(r'(\d+)\s*(?:MB|Mo)\s*(?:cache|L\d)?', text, re.I)
    specs["Cache (MB)"] = match.group(1) if match else None

    # Socket
    match = re.search(r'(AM\d+|LGA\s*\d+)', text, re.I)
    specs["Socket"] = match.group(1).replace(" ", "") if match else None

    return specs


async def scrape_amazon_cpus():
    start_url = "https://www.amazon.fr/-/en/gp/browse.html?rw_useCurrentProtocol=1&node=430352031&ref_=amb_link_sj7jvnB9SQalv7yCEAD-TA_28"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        current_url = start_url
        while True:
            print(f"\n Loading page {page_number}: {current_url}")
            await page.goto(current_url)
            try:
                await page.wait_for_selector("div[data-cy='title-recipe']", timeout=25000)
            except:
                print("No products found. Stopping.")
                break

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            products = soup.select("div[data-cy='title-recipe']")

            for product in tqdm(products, desc=f"Extracting page {page_number}"):
                link_tag = product.select_one("a.a-link-normal")
                title_tag = product.select_one("a h2 span")

                if not title_tag or not link_tag:
                    continue

                name = title_tag.get_text(strip=True)
                link = "https://www.amazon.fr" + link_tag["href"]

                # Try to find a secondary line (subtitle/specs)
                sub_text = ""
                subtitle_tag = product.find_next("div", attrs={"data-cy": "secondary-description-recipe"})
                if subtitle_tag:
                    sub_text = subtitle_tag.get_text(strip=True)

                full_text = f"{name} {sub_text}"

                # Find price in same container
                container = product.find_parent("div", class_="puisg-col-inner")
                price_tag = None
                if container:
                    price_tag = container.select_one("div[data-cy='price-recipe'] span.a-offscreen")
                price = price_tag.get_text(strip=True) if price_tag else "N/A"

                specs = extract_specs(full_text)
                all_data.append({
                    "Name": name,
                    "URL": link,
                    "Price": price,
                    **specs
                })

            # Pagination
            next_button = soup.select_one("a.s-pagination-next")
            if next_button and "href" in next_button.attrs and "disabled" not in next_button.get("class", []):
                next_href = next_button["href"]
                current_url = "https://www.amazon.fr" + next_href
                page_number += 1
                await asyncio.sleep(2)
            else:
                print("No more pages found.")
                break

        await browser.close()

    # Clean and export
    df = pd.DataFrame(all_data)
    df.drop_duplicates(subset=["Name"], inplace=True)
    df.to_csv("amazon_cpus.csv", index=False)
    print(f"\nScraping complete! {len(df)} CPUs saved to amazon_cpus.csv")


await scrape_amazon_cpus()

CancelledError: 

# **GPUs**

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re # Import the re module for regex
from tqdm.asyncio import tqdm
import time

# === SMART SPEC EXTRACTOR ===
def extract_specs(text):
    text = text.replace(",", ".").replace("-", " ")
    specs = {}

    # Cores
    match = re.search(r'(\d+)\s*(?:core|coeur|coeurs|cores?|C)\b', text, re.I)
    specs["Cores"] = match.group(1) if match else None

    # Threads
    match = re.search(r'(\d+)\s*(?:thread|threads|T)\b', text, re.I)
    specs["Threads"] = match.group(1) if match else None

    # Frequency (GHz)
    match = re.search(r'(\d+(?:\.\d+)?)\s*(?:GHz|GHZ)', text, re.I)
    specs["Frequency (GHz)"] = match.group(1) if match else None

    # Cache (MB)
    match = re.search(r'(\d+)\s*(?:MB|Mo)\s*(?:cache|L\d)?', text, re.I)
    specs["Cache (MB)"] = match.group(1) if match else None

    # Socket
    match = re.search(r'(AM\d+|LGA\s*\d+)', text, re.I)
    specs["Socket"] = match.group(1).replace(" ", "") if match else None

    return specs


async def scrape_amazon_gpus():
    start_url = "https://www.amazon.fr/-/en/gp/browse.html?rw_useCurrentProtocol=1&node=430340031&ref_=amb_link_sj7jvnB9SQalv7yCEAD-TA_29"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        current_url = start_url
        while True:
            print(f"\n Loading page {page_number}: {current_url}")
            await page.goto(current_url)
            try:
                await page.wait_for_selector("div[data-cy='title-recipe']", timeout=25000)
            except:
                print(" No products found. Stopping.")
                break

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            products = soup.select("div[data-cy='title-recipe']")

            for product in tqdm(products, desc=f"Extracting GPUs page {page_number}"):
                # Product Title & URL
                link_tag = product.select_one("a.a-link-normal")
                title_tag = product.select_one("a h2 span")
                if not title_tag or not link_tag:
                    continue

                name = title_tag.get_text(strip=True)
                link = "https://www.amazon.fr" + link_tag["href"]

                # Find parent container
                container = product.find_parent("div", class_="puisg-col-inner")
                if not container:
                    continue

                # Price
                price_tag = container.select_one("div[data-cy='price-recipe'] span.a-offscreen")
                price = price_tag.get_text(strip=True) if price_tag else "N/A"

                # Specs block
                specs_block = container.select_one("div[data-cy='product-details-recipe']")
                specs = {}
                if specs_block:
                    for spec in specs_block.select("div.puisg-col-inner"):
                        label_tag = spec.select_one("span.a-color-secondary")
                        value_tag = spec.select_one("span.a-text-bold")
                        if label_tag and value_tag:
                            key = label_tag.get_text(strip=True)
                            val = value_tag.get_text(strip=True)
                            specs[key] = val

                # Merge all data
                data = {
                    "Name": name,
                    "URL": link,
                    "Price": price,
                }
                data.update(specs)
                all_data.append(data)

            # Pagination handling
            next_button = soup.select_one("a.s-pagination-next")
            if next_button and "href" in next_button.attrs and "disabled" not in next_button.get("class", []):
                next_href = next_button["href"]
                current_url = "https://www.amazon.fr" + next_href
                page_number += 1
                await asyncio.sleep(2) # small delay to avoid being blocked
            else:
                print(" No more pages found.")
                break

        await browser.close()

    # Export to CSV
    df = pd.DataFrame(all_data)
    df.drop_duplicates(subset=["Name"], inplace=True)
    df.to_csv("amazon_gpus.csv", index=False)
    print(f"\n Scraping complete! {len(df)} graphics cards saved to amazon_gpus.csv")

# Run
await scrape_amazon_gpus()


🕐 Loading page 1: https://www.amazon.fr/-/en/gp/browse.html?rw_useCurrentProtocol=1&node=430340031&ref_=amb_link_sj7jvnB9SQalv7yCEAD-TA_29


Extracting GPUs page 1: 100%|██████████| 24/24 [00:00<00:00, 769.33it/s]



🕐 Loading page 2: https://www.amazon.fr/s?k=Graphics+Cards&i=computers&rh=n%3A430340031&page=2&xpid=-8Th6JWl_oz2l&c=ts&qid=1760138436&ts_id=430340031&ref=sr_pg_1


Extracting GPUs page 2: 100%|██████████| 24/24 [00:00<00:00, 433.05it/s]



🕐 Loading page 3: https://www.amazon.fr/s?k=Graphics+Cards&i=computers&rh=n%3A430340031&page=3&c=ts&qid=1760138443&ts_id=430340031&xpid=-8Th6JWl_oz2l&ref=sr_pg_2


...


🕐 Loading page 21: https://www.amazon.fr/s?k=Graphics+Cards&i=computers&rh=n%3A430340031&page=21&c=ts&qid=1760138538&ts_id=430340031&xpid=-8Th6JWl_oz2l&ref=sr_pg_20


Extracting GPUs page 21: 100%|██████████| 24/24 [00:00<00:00, 445.25it/s]



🕐 Loading page 22: https://www.amazon.fr/s?k=Graphics+Cards&i=computers&rh=n%3A430340031&page=22&c=ts&qid=1760138543&ts_id=430340031&xpid=-8Th6JWl_oz2l&ref=sr_pg_21


Extracting GPUs page 22: 100%|██████████| 15/15 [00:00<00:00, 508.79it/s]

✅ No more pages found.

✅ Scraping complete! 493 graphics cards saved to amazon_gpus.csv


# **Motherboards**

In [ ]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.asyncio import tqdm # Use async tqdm
import time

def extract_mobo_specs(title: str):
    """Extract motherboard specs from Amazon title with more precision."""
    specs = {}
    t = title.replace(",", " ").replace("/", " ").replace("-", " ").lower()

    # Brand
    m = re.search(r'\b(asus|msi|gigabyte|asrock|biostar|nzxt)\b', t)
    specs["Brand"] = m.group(0).capitalize() if m else None

    # Chipset (X870E, B650, Z790, H670, etc.)
    m = re.search(r'\b([bxzh]?\d{3,4}[a-z]?)\b', t)
    specs["Chipset"] = m.group(0).upper() if m else None

    # CPU Socket
    m = re.search(r'\b(am\d|lga\d+)\b', t)
    specs["Socket"] = m.group(0).upper() if m else None

    # Form Factor
    m = re.search(r'\b(micro ?atx|mini ?itx|atx|e ?atx)\b', t)
    specs["Form Factor"] = m.group(0).replace(" ", "").upper() if m else None

    # Memory Type + optional OC speed
    mem_match = re.search(r'\b(ddr4|ddr5)(?:\s*[\w+-]*\s*(\d{3,5})?mt/s)?', t)
    if mem_match:
        specs["Memory Type"] = mem_match.group(1).upper()
        specs["Memory Max OC"] = mem_match.group(2) if mem_match.group(2) else None
    else:
        specs["Memory Type"] = None
        specs["Memory Max OC"] = None

    # Wi-Fi standard
    wifi_match = re.search(r'wifi\s*([678]?e?)', t)
    specs["WiFi"] = "Yes" if wifi_match else "No"
    specs["WiFi Standard"] = "WiFi " + wifi_match.group(1) if wifi_match else None

    # PCIe Gen (handle multiple slots)
    pcie_match = re.findall(r'pcie\s*([0-9](?:\.[0-9])?)', t)
    specs["PCIe"] = ", ".join(pcie_match) if pcie_match else None

    # M.2 support / Gen
    m2_match = re.findall(r'm\.?2\s*gen\s*([0-9])', t)
    specs["M2 Gen"] = ", ".join(m2_match) if m2_match else None

    # VRM / other optional info (like 110A)
    vrm_match = re.search(r'vrm\s*(\d{2,3}a?)', t)
    specs["VRM"] = vrm_match.group(1).upper() if vrm_match else None

    return specs


async def scrape_motherboards(max_pages=75): # Adjusted max_pages to match the call
    start_url = "https://www.amazon.fr/-/en/gp/browse.html?rw_useCurrentProtocol=1&node=430341031&ref_=amb_link_sj7jvnB9SQalv7yCEAD-TA_31"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        current_url = start_url

        while page_number <= max_pages:
            print(f"\n🕐 Loading page {page_number}: {current_url}")
            try:
                await page.goto(current_url)
            except Exception as e:
                print(f" Failed to load page {page_number}: {e}")
                break

            try:
                await page.wait_for_selector("div[data-cy='title-recipe']", timeout=20000)
            except PlaywrightTimeoutError:
                print(" Timeout waiting for product titles. Stopping.")
                break

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            products = soup.select("div[data-cy='title-recipe']")

            if not products:
                print(" No products found on this page. Possibly end.")
                break

            for product in tqdm(products, desc=f"Extracting page {page_number}"):
                link_tag = product.select_one("a.a-link-normal")
                title_tag = product.select_one("a h2 span")
                if not link_tag or not title_tag:
                    continue

                name = title_tag.get_text(strip=True)
                link = "https://www.amazon.fr" + link_tag.get("href", "")

                # Price
                container = product.find_parent("div", class_="puisg-col-inner")
                price = "N/A"
                if container:
                    price_tag = container.select_one("div[data-cy='price-recipe'] span.a-offscreen")
                    if price_tag:
                        price = price_tag.get_text(strip=True)

                specs = extract_mobo_specs(name)

                row = {
                    "Name": name,
                    "URL": link,
                    "Price": price,
                    **specs
                }
                all_data.append(row)

            # Pagination – find “Next” button
            next_btn = soup.select_one("a.s-pagination-next")
            if next_btn and next_btn.has_attr("href") and "disabled" not in next_btn.get("class", []):
                next_href = next_btn["href"]
                current_url = "https://www.amazon.fr" + next_href
                page_number += 1
                await asyncio.sleep(1 + (page_number % 2))  # small variable delay, use await
            else:
                print("No next page found or disabled. Done.")
                break

        await browser.close()

    df = pd.DataFrame(all_data)
    df.drop_duplicates(subset=["Name"], inplace=True)
    out_file = "amazon_motherboards_all.csv"
    df.to_csv(out_file, index=False)
    print(f"\n Scraping done. {len(df)} motherboards saved to {out_file}")

# Run the scraper
await scrape_motherboards(max_pages=75) # Use await


🕐 Loading page 1: https://www.amazon.fr/-/en/gp/browse.html?rw_useCurrentProtocol=1&node=430341031&ref_=amb_link_sj7jvnB9SQalv7yCEAD-TA_31


Extracting page 1: 100%|██████████| 24/24 [00:00<00:00, 2566.44it/s]



🕐 Loading page 2: https://www.amazon.fr/s?k=Motherboards&i=computers&rh=n%3A430341031&page=2&xpid=ZI83ORZ8tsFrB&c=ts&qid=1760141076&ts_id=430341031&ref=sr_pg_1


Extracting page 2: 100%|██████████| 24/24 [00:00<00:00, 289.77it/s]



🕐 Loading page 3: https://www.amazon.fr/s?k=Motherboards&i=computers&rh=n%3A430341031&page=3&c=ts&qid=1760141081&ts_id=430341031&xpid=ZI83ORZ8tsFrB&ref=sr_pg_2


...


🕐 Loading page 74: https://www.amazon.fr/s?k=Motherboards&i=computers&rh=n%3A430341031&page=74&c=ts&qid=1760141385&ts_id=430341031&xpid=ZI83ORZ8tsFrB&ref=sr_pg_73


Extracting page 74: 100%|██████████| 24/24 [00:00<00:00, 1943.61it/s]



🕐 Loading page 75: https://www.amazon.fr/s?k=Motherboards&i=computers&rh=n%3A430341031&page=75&c=ts&qid=1760141389&ts_id=430341031&xpid=ZI83ORZ8tsFrB&ref=sr_pg_74


Extracting page 75: 100%|██████████| 24/24 [00:00<00:00, 2664.32it/s]



✅ Scraping done. 1698 motherboards saved to amazon_motherboards_all.csv


# **RAM**

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# --- Function to extract RAM specs from product title ---
def extract_ram_specs(title: str):
    specs = {}
    t = title.lower()

    # Brand
    m = re.search(r'\b(crucial|corsair|g\.skill|kingston|patriot|teamgroup)\b', t)
    specs["Brand"] = m.group(0).capitalize() if m else None

    # Model
    m = re.search(r'\b([a-z0-9]+)\s+pro\s+ram\b', t) # This regex might need refinement based on actual titles
    specs["Model"] = m.group(1).upper() if m else None

    # Total Capacity
    m = re.search(r'(\d{2,4})gb', t)
    specs["Total Capacity"] = m.group(1) + "GB" if m else None

    # Kit / Module
    m = re.search(r'\((\d+x\d+gb)\)', t)
    specs["Kit"] = m.group(1) if m else None

    # Speed
    m = re.search(r'(\d{3,5})mhz', t)
    specs["Speed"] = m.group(1) + "MHz" if m else None

    # CAS Latency
    m = re.search(r'cl(\d+)', t)
    specs["CAS Latency"] = "CL" + m.group(1) if m else None

    # Type
    if "ddr4" in t:
        specs["Type"] = "DDR4"
    elif "ddr5" in t or (specs.get("Speed") and int(specs["Speed"].replace("MHz","")) >= 4800):
        specs["Type"] = "DDR5"
    else:
        specs["Type"] = None

    # Features
    features = []
    if "xmp" in t:
        features.append("Intel XMP")
    if "expo" in t:
        features.append("AMD EXPO")
    if "overclock" in t:
        features.append("Overclocking")
    specs["Features"] = ", ".join(features) if features else None

    return specs

# --- Function to parse products from one page ---
async def parse_amazon_page(page):
    products = []
    # select product blocks - using a more specific selector
    divs = await page.query_selector_all("div[data-cy='title-recipe']")
    for div in divs:
        title_el = await div.query_selector("h2")
        # Find parent container to locate price, rating, reviews
        container = await div.query_selector("xpath=..") # Find parent div
        price_el = await container.query_selector("span.a-price") if container else None
        rating_el = await container.query_selector("i.a-icon-star-mini") if container else None
        reviews_el = await container.query_selector("a[href*='customerReviews']") if container else None


        if title_el:
            title = await title_el.inner_text()
            specs = extract_ram_specs(title) # This function is not async
            specs["Title"] = title
            specs["Price"] = await price_el.inner_text() if price_el else None
            specs["Rating"] = await rating_el.get_attribute("aria-label") if rating_el else None
            specs["Reviews"] = await reviews_el.inner_text() if reviews_el else None
            products.append(specs)
    return products

# --- Main scraping loop ---
async def scrape_amazon_ram(max_pages=75):
    all_products = []
    base_url = "https://www.amazon.fr/-/en/gp/browse.html?rw_useCurrentProtocol=1&node=430351031&page={page}"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True) # headless=True for no browser window
        page = await browser.new_page()
        page.set_default_timeout(60000)

        for page_num in range(1, max_pages + 1):
            print(f"Scraping page {page_num}...")
            url = base_url.format(page=page_num)
            try:
                await page.goto(url, timeout=60000)
            except Exception as e:
                print(f"Error loading page {page_num}: {e}")
                continue

            # Wait for product titles to load
            try:
                 await page.wait_for_selector("div[data-cy='title-recipe']", timeout=20000)
            except Exception as e:
                 print(f"No products found on page {page_num}. Stopping.")
                 break # Stop if no products are found

            products_on_page = await parse_amazon_page(page) # Use await
            all_products.extend(products_on_page)
            await asyncio.sleep(1 + (page_num % 2)) # small variable delay

        await browser.close()

    # --- Save results ---
    df = pd.DataFrame(all_products)
    df.to_csv("amazon_ram_playwright.csv", index=False)
    print("Scraping complete! Saved to amazon_ram_playwright.csv")

# Run the scraper
await scrape_amazon_ram(max_pages=75) # Adjust max_pages as needed

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

# **LDLC Scraping**

# **CPUs**

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
import time


# ---------------------- SPEC EXTRACTOR ----------------------
def extract_specs(text):
    text = text.replace(",", ".").replace("-", " ")
    specs = {}

    # Cores
    match = re.search(r'(\d+)\s*(?:core|coeur|coeurs|cores?)\b', text, re.I)
    specs["Cores"] = match.group(1) if match else None

    # Threads
    match = re.search(r'(\d+)\s*(?:thread|threads|T)\b', text, re.I)
    specs["Threads"] = match.group(1) if match else None

    # Frequency (GHz)
    match = re.search(r'(\d+(?:\.\d+)?)\s*(?:GHz|GHZ)', text, re.I)
    specs["Frequency (GHz)"] = match.group(1) if match else None

    # Cache (MB)
    match = re.search(r'(\d+)\s*(?:MB|Mo)\s*(?:cache|L\d)?', text, re.I)
    specs["Cache (MB)"] = match.group(1) if match else None

    # Socket
    match = re.search(r'(AM\d+|LGA\s*\d+)', text, re.I)
    specs["Socket"] = match.group(1).replace(" ", "") if match else None

    return specs


# ---------------------- SCRAPER FUNCTION ----------------------
async def scrape_ldlc_cpus():
    start_url = "https://www.ldlc.com/informatique/pieces-informatique/processeur/c4300"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        current_url = start_url
        while True:
            print(f"\n Loading page {page_number}: {current_url}")
            await page.goto(current_url)
            await page.wait_for_load_state("networkidle")

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            products = soup.select("li.pdt-item")

            if not products:
                print("No more products found.")
                break

            for item in products:
                name_tag = item.select_one("h3.title-3 a")
                desc_tag = item.select_one("p.desc")
                price_tag = item.select_one("div.price")

                name = name_tag.get_text(strip=True) if name_tag else None
                desc = desc_tag.get_text(strip=True) if desc_tag else ""
                price = price_tag.get_text(strip=True) if price_tag else "N/A"
                link = (
                    "https://www.ldlc.com" + name_tag["href"]
                    if name_tag and name_tag.has_attr("href")
                    else None
                )

                # Extract structured specs
                specs = extract_specs(f"{name} {desc}")

                all_data.append({
                    "Name": name,
                    "Description": desc,
                    "Price": price,
                    "URL": link,
                    **specs
                })

            # Try to find pagination button
            next_button = soup.select_one("a.page-suivante")
            if next_button and "href" in next_button.attrs:
                next_url = "https://www.ldlc.com" + next_button["href"]
                current_url = next_url
                page_number += 1
                time.sleep(2)
            else:
                print(" All pages scraped.")
                break

        await browser.close()

    # Save results
    df = pd.DataFrame(all_data)
    df.drop_duplicates(subset=["Name"], inplace=True)
    df.to_csv("ldlc_cpus.csv", index=False)
    print(f"\n Scraping complete! {len(df)} CPUs saved to ldlc_cpus.csv.")


# ---------------------- RUN ----------------------
await scrape_ldlc_cpus()



🔄 Loading page 1: https://www.ldlc.com/informatique/pieces-informatique/processeur/c4300
✅ All pages scraped.

💾 Scraping complete! 44 CPUs saved to ldlc_cpus.csv.


# **GPUs**

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
import time


# ---------------------- SPEC EXTRACTOR ----------------------
def extract_gpu_specs(text):
    text = text.replace(",", ".").replace("-", " ").lower()
    specs = {}

    # Memory (Go / GB)
    match = re.search(r'(\d+)\s*(?:go|gb)\s*(?:gddr\d*)?', text, re.I)
    specs["Memory (GB)"] = match.group(1) if match else None

    # Memory Type
    match = re.search(r'(gddr\d+)', text, re.I)
    specs["Memory Type"] = match.group(1).upper() if match else None

    # Chipset (RTX, GTX, Radeon, etc.)
    match = re.search(r'(rtx\s*\d{3,4}|gtx\s*\d{3,4}|rx\s*\d{3,4}|arc\s*\w+)', text, re.I)
    specs["Chipset"] = match.group(1).upper().replace(" ", "") if match else None

    # Bus / PCI Express
    match = re.search(r'(pci\s*express\s*\d*\.?\d*)', text, re.I)
    specs["Bus Interface"] = match.group(1).upper().replace(" ", "") if match else None

    # Ports
    ports = []
    if re.search(r'hdmi', text, re.I): ports.append("HDMI")
    if re.search(r'displayport', text, re.I): ports.append("DisplayPort")
    if re.search(r'dvi', text, re.I): ports.append("DVI")
    if ports:
        specs["Ports"] = ", ".join(ports)

    # Cooling type
    match = re.search(r'(ventilateur|blower|fan|watercooling|radiateur)', text, re.I)
    specs["Cooling"] = match.group(1).capitalize() if match else None

    return specs


# ---------------------- SCRAPER FUNCTION ----------------------
async def scrape_ldlc_gpus():
    start_url = "https://www.ldlc.com/informatique/pieces-informatique/carte-graphique-interne/c4684"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        current_url = start_url
        while True:
            print(f"\n Loading page {page_number}: {current_url}")
            await page.goto(current_url)
            await page.wait_for_load_state("networkidle")

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            products = soup.select("li.pdt-item")

            if not products:
                print(" No more products found.")
                break

            for item in products:
                name_tag = item.select_one("h3.title-3 a")
                desc_tag = item.select_one("p.desc")
                price_tag = item.select_one("div.price")

                name = name_tag.get_text(strip=True) if name_tag else None
                desc = desc_tag.get_text(strip=True) if desc_tag else ""
                price = price_tag.get_text(strip=True) if price_tag else "N/A"
                link = (
                    "https://www.ldlc.com" + name_tag["href"]
                    if name_tag and name_tag.has_attr("href")
                    else None
                )

                # Structured GPU specs
                specs = extract_gpu_specs(f"{name} {desc}")

                all_data.append({
                    "Name": name,
                    "Description": desc,
                    "Price": price,
                    "URL": link,
                    **specs
                })

            # Try to find pagination button
            next_button = soup.select_one("a.page-suivante")
            if next_button and "href" in next_button.attrs:
                next_url = "https://www.ldlc.com" + next_button["href"]
                current_url = next_url
                page_number += 1
                time.sleep(2)
            else:
                print("\n All pages scraped.")
                break

        await browser.close()

    # Save results
    df = pd.DataFrame(all_data)
    df.drop_duplicates(subset=["Name"], inplace=True)
    df.to_csv("ldlc_gpus.csv", index=False)
    print(f"\n Scraping complete! {len(df)} GPUs saved to ldlc_gpus.csv.")


# ---------------------- RUN ----------------------
await scrape_ldlc_gpus()



🧭 Loading page 1: https://www.ldlc.com/informatique/pieces-informatique/carte-graphique-interne/c4684

✅ All pages scraped.

💾 Scraping complete! 48 GPUs saved to ldlc_gpus.csv.


# **MotherBoards**

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# ---------------------- SPEC EXTRACTOR ----------------------
def extract_motherboard_specs(text):
    text = text.replace(",", ".").replace("-", " ")
    specs = {}

    # Socket
    match = re.search(r'(LGA\s*\d+|AM\d+|sTRX\d+|s\d+)', text, re.I)
    specs["Socket"] = match.group(1).replace(" ", "") if match else None

    # Chipset
    match = re.search(r'\b(B\d{3}|Z\d{3}|H\d{3}|A\d{3}|X\d{3}|A520|B550|X570|H610|B760|Z790)\b', text, re.I)
    specs["Chipset"] = match.group(1).upper() if match else None

    # RAM Type
    match = re.search(r'(DDR[3-5])', text, re.I)
    specs["RAM Type"] = match.group(1).upper() if match else None

    # Form Factor
    match = re.search(r'(ATX|Micro\s*ATX|Mini\s*ITX)', text, re.I)
    specs["Form Factor"] = match.group(1).replace(" ", "") if match else None

    # M.2 Slots
    match = re.search(r'(\d+)\s*x?\s*M\.2', text, re.I)
    specs["M.2 Slots"] = match.group(1) if match else None

    # PCIe version
    match = re.search(r'(PCI[-\s]*Express\s*(\d+(\.\d+)?))', text, re.I)
    specs["PCIe"] = match.group(1).replace(" ", "") if match else None

    # USB version
    match = re.search(r'(USB\s*3\.2|USB\s*3\.1|USB\s*3\.0|USB\s*2\.0)', text, re.I)
    specs["USB"] = match.group(1).replace(" ", "") if match else None

    return specs


# ---------------------- SCRAPER FUNCTION ----------------------
async def scrape_ldlc_motherboards():
    start_url = "https://www.ldlc.com/informatique/pieces-informatique/carte-mere/c4293"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        current_url = start_url
        while True:
            print(f"\n Loading page {page_number}: {current_url}")
            try:
                await page.goto(current_url)
                await page.wait_for_load_state("networkidle")
            except Exception as e:
                print(f"Timeout or error: {e}")
                break

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            products = soup.select("li.pdt-item")

            if not products:
                print("No more products found.")
                break

            for item in products:
                name_tag = item.select_one("h3.title-3 a")
                desc_tag = item.select_one("p.desc")
                price_tag = item.select_one("div.price")

                name = name_tag.get_text(strip=True) if name_tag else None
                desc = desc_tag.get_text(strip=True) if desc_tag else ""
                price = price_tag.get_text(strip=True).replace("\u202f", "") if price_tag else "N/A"
                link = (
                    "https://www.ldlc.com" + name_tag["href"]
                    if name_tag and name_tag.has_attr("href")
                    else None
                )

                specs = extract_motherboard_specs(f"{name} {desc}")

                all_data.append({
                    "Name": name,
                    "Description": desc,
                    "Price": price,
                    "URL": link,
                    **specs
                })

            # Check pagination
            next_button = soup.select_one("a.page-suivante")
            if next_button and "href" in next_button.attrs:
                next_url = "https://www.ldlc.com" + next_button["href"]
                current_url = next_url
                page_number += 1
                time.sleep(2)
            else:
                print("All pages scraped.")
                break

        await browser.close()

    # Save results
    df = pd.DataFrame(all_data)
    df.drop_duplicates(subset=["Name"], inplace=True)
    df.to_csv("ldlc_motherboards.csv", index=False)
    print(f"\n Scraping complete! {len(df)} motherboards saved to ldlc_motherboards.csv.")


# ---------------------- RUN ----------------------
await scrape_ldlc_motherboards()



🧭 Loading page 1: https://www.ldlc.com/informatique/pieces-informatique/carte-mere/c4293
✅ All pages scraped.

💾 Scraping complete! 48 motherboards saved to ldlc_motherboards.csv.


# **Ram**

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# ---------------------- SPEC EXTRACTOR ----------------------
def extract_ram_specs(text):
    text = text.replace(",", ".").replace("-", " ").lower()
    specs = {}

    # Capacity (GB)
    match = re.search(r'(\d+)\s*(go|gb)', text, re.I)
    specs["Capacity (GB)"] = match.group(1) if match else None

    # Number of sticks / modules
    match = re.search(r'\(?(\d+)\s*x\s*\d+\s*go\)?', text, re.I)
    specs["Modules"] = match.group(1) if match else "1"

    # RAM type (DDR3/DDR4/DDR5)
    match = re.search(r'(ddr[3-5])', text, re.I)
    specs["Type"] = match.group(1).upper() if match else None

    # Frequency (MHz)
    match = re.search(r'(\d{3,5})\s*mhz', text, re.I)
    specs["Frequency (MHz)"] = match.group(1) if match else None

    # CAS Latency (CL)
    match = re.search(r'cl(\d+)', text, re.I)
    specs["CAS Latency"] = match.group(1) if match else None

    # RGB
    specs["RGB"] = "Yes" if re.search(r'rgb', text, re.I) else "No"

    return specs


# ---------------------- SCRAPER FUNCTION ----------------------
async def scrape_ldlc_ram():
    start_url = "https://www.ldlc.com/informatique/pieces-informatique/memoire-pc/c4703"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        current_url = start_url
        while True:
            print(f"\n Loading page {page_number}: {current_url}")
            try:
                await page.goto(current_url)
                await page.wait_for_load_state("networkidle")
            except Exception as e:
                print(f" Timeout or error: {e}")
                break

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            products = soup.select("li.pdt-item")

            if not products:
                print(" No products found on this page.")
                break

            print(f" Found {len(products)} products on page {page_number}")

            for item in products:
                name_tag = item.select_one("h3.title-3 a")
                desc_tag = item.select_one("p.desc")
                price_tag = item.select_one("div.price")

                name = name_tag.get_text(strip=True) if name_tag else None
                desc = desc_tag.get_text(strip=True) if desc_tag else ""
                price = price_tag.get_text(strip=True).replace("\u202f", "") if price_tag else "N/A"
                link = (
                    "https://www.ldlc.com" + name_tag["href"]
                    if name_tag and name_tag.has_attr("href")
                    else None
                )

                specs = extract_ram_specs(f"{name} {desc}")

                all_data.append({
                    "Name": name,
                    "Description": desc,
                    "Price": price,
                    "URL": link,
                    **specs
                })

            # Check for next page
            next_button = soup.select_one("a.page-suivante")
            if next_button and "href" in next_button.attrs:
                next_url = "https://www.ldlc.com" + next_button["href"]
                current_url = next_url
                page_number += 1
                time.sleep(2)
            else:
                print(" All pages scraped.")
                break

        await browser.close()

    # Save all data
    df = pd.DataFrame(all_data)
    df.drop_duplicates(subset=["Name"], inplace=True)
    df.to_csv("ldlc_ram.csv", index=False)
    print(f"\n Scraping complete! {len(df)} RAM modules saved to ldlc_ram.csv.")


# ---------------------- RUN ----------------------
await scrape_ldlc_ram()



🧭 Loading page 1: https://www.ldlc.com/informatique/pieces-informatique/memoire-pc/c4703
✅ Found 48 products on page 1
✅ All pages scraped.

💾 Scraping complete! 45 RAM modules saved to ldlc_ram.csv.


# Neweeg

# **CPUs**

In [6]:
!pip install playwright
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.12/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

# **motherboards**

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.asyncio import tqdm
import time

# MOTHERBOARD SPEC EXTRACTOR
def extract_motherboard_specs(text):
    text = text.replace(",", ".").replace("-", " ").lower()
    specs = {}

    # Socket
    socket_match = re.search(r'(am\d+|lga\s*\d+|socket\s*\w+)', text, re.I)
    specs["Socket"] = socket_match.group(1).replace(" ", "").upper() if socket_match else None

    # Chipset
    chipset_match = re.search(r'(b\d+|x\d+|z\d+|h\d+|a\d+)(?:\s*\w*)?', text, re.I)
    specs["Chipset"] = chipset_match.group(1).upper() if chipset_match else None

    # Form Factor
    form_factors = ["atx", "micro atx", "mini itx", "extended atx", "m-atx", "itx"]
    for ff in form_factors:
        if ff in text:
            specs["Form Factor"] = ff.upper()
            break

    # Memory Type
    memory_match = re.search(r'(ddr\d+)', text, re.I)
    specs["Memory Type"] = memory_match.group(1).upper() if memory_match else None

    # M.2 Slots
    m2_match = re.search(r'(\d+)\s*x?\s*m\.?2', text, re.I)
    specs["M.2 Slots"] = m2_match.group(1) if m2_match else None

    # PCIe Version
    pcie_match = re.search(r'pcie\s*(\d+\.?\d?)', text, re.I)
    specs["PCIe Version"] = pcie_match.group(1) if pcie_match else None

    # WiFi
    wifi_match = re.search(r'(wifi\s*\d?\w?|wireless\s*\w+)', text, re.I)
    specs["WiFi"] = wifi_match.group(1).upper() if wifi_match else None

    # LAN Speed
    lan_match = re.search(r'(\d+\.?\d?)\s*gb(e|)\s*lan', text, re.I)
    specs["LAN Speed"] = lan_match.group(1) + "Gb" if lan_match else None

    return specs

async def scrape_newegg_motherboards():
    start_url = "https://www.newegg.com/p/pl?N=100006654"
    all_data = []
    page_number = 1

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Set user agent to avoid blocking
        await page.set_extra_http_headers({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })

        page.set_default_timeout(60000)

        current_url = start_url
        while page_number <= 3:  # Limit to 3 pages for demo, remove for full scrape
            print(f"\nLoading page {page_number}: {current_url}")

            try:
                await page.goto(current_url, wait_until='networkidle')
                await page.wait_for_selector(".item-cell", timeout=30000)
            except Exception as e:
                print(f"Error loading page: {e}")
                break

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")

            # Find all motherboard items
            products = soup.select(".item-cell")
            print(f"Found {len(products)} products on page {page_number}")

            for product in tqdm(products, desc=f"Extracting page {page_number}"):
                try:
                    # Extract product name
                    title_tag = product.select_one(".item-title")
                    if not title_tag:
                        continue

                    name = title_tag.get_text(strip=True)

                    # Extract product URL
                    link_tag = product.select_one(".item-title")
                    if link_tag and link_tag.get('href'):
                        link = link_tag['href']
                        if not link.startswith('http'):
                            link = "https://www.newegg.com" + link
                    else:
                        link = "N/A"

                    # Extract price
                    price_tag = product.select_one(".price-current strong")
                    price_decimal = product.select_one(".price-current sup")

                    price = "N/A"
                    if price_tag:
                        price = f"${price_tag.get_text(strip=True)}"
                        if price_decimal:
                            price += f".{price_decimal.get_text(strip=True)}"

                    # Extract brand
                    brand_tag = product.select_one(".item-brand img")
                    brand = brand_tag.get('title') if brand_tag else "N/A"

                    # Extract rating
                    rating_tag = product.select_one(".item-rating")
                    rating = "N/A"
                    rating_count = "N/A"
                    if rating_tag:
                        rating_span = rating_tag.select_one(".item-rating-num")
                        if rating_span:
                            rating_count = rating_span.get_text(strip=True).strip('()')

                    # Extract shipping info
                    shipping_tag = product.select_one(".product-price-ship-eligible")
                    shipping = shipping_tag.get_text(strip=True) if shipping_tag else "N/A"

                    # Combine text for spec extraction
                    description_text = f"{name} {brand}"

                    # Extract features list
                    features_tag = product.select_one(".item-features")
                    features_text = ""
                    if features_tag:
                        features_items = features_tag.find_all("li")
                        for item in features_items:
                            features_text += " " + item.get_text(strip=True)

                    full_text = f"{description_text} {features_text}"

                    specs = extract_motherboard_specs(full_text)

                    all_data.append({
                        "Name": name,
                        "Brand": brand,
                        "Price": price,
                        "URL": link,
                        "Rating Count": rating_count,
                        "Shipping": shipping,
                        **specs
                    })

                except Exception as e:
                    print(f"Error extracting product: {e}")
                    continue

            # Pagination - find next page
            next_button = soup.select_one("a.btn[title='Next']")
            if next_button and "href" in next_button.attrs and "is-disabled" not in next_button.get("class", []):
                next_href = next_button["href"]
                current_url = next_href
                page_number += 1
                await asyncio.sleep(2)  # Be respectful with delays
            else:
                print("No more pages found.")
                break

        await browser.close()

    # Clean and export
    if all_data:
        df = pd.DataFrame(all_data)
        df.drop_duplicates(subset=["Name"], inplace=True)
        df.to_csv("newegg_motherboards.csv", index=False)
        print(f"\nScraping complete! {len(df)} motherboards saved to newegg_motherboards.csv")

        # Display sample of data
        print("\nSample of scraped data:")
        print(df.head()[["Name", "Brand", "Price", "Socket", "Chipset"]])
    else:
        print("No data was scraped.")

# Run the scraper
if __name__ == "__main__":
    asyncio.run(scrape_newegg_motherboards())


Loading page 1: https://www.newegg.com/p/pl?N=100006654
Found 36 products on page 1


Extracting page 1: 100%|██████████| 36/36 [00:00<00:00, 269.04it/s]



Loading page 2: https://www.newegg.com/p/pl?N=100006654&page=2
Error loading page: Page.goto: Timeout 60000ms exceeded.
Call log:
  - navigating to "https://www.newegg.com/p/pl?N=100006654&page=2", waiting until "networkidle"


Scraping complete! 36 motherboards saved to newegg_motherboards.csv

Sample of scraped data:
                                                Name     Brand     Price  \
0  MSI MAG X870 TOMAHAWK WIFI AM5 AMD X870 ATX Mo...       MSI  $309..99   
1  GIGABYTE B650M GAMING PLUS WIFI AM5 LGA 1718 A...  GIGABYTE  $129..99   
2  GIGABYTE B550M GAMING X WIFI6 AM4 AMD B550 Mic...  GIGABYTE  $104..99   
3  ASUS B850 MAX GAMING WIFI W - AMD AM5 ATX Moth...      ASUS  $169..99   
4  ASRock B650M Pro RS WiFi AM5 AMD B650 Micro AT...    ASRock  $129..99   

  Socket Chipset  
0    AM5    X870  
1    AM5    B650  
2    AM4    B550  
3    AM5    B850  
4    AM5    B650  


# **Ram**

In [8]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError # Import TimeoutError
import pandas as pd
import time # Import time

async def scrape_newegg_ram():
    base_url = "https://www.newegg.com/p/pl?N=100006650&page="
    all_products = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                          "(KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
        })

        # Increase default timeout to 60 seconds
        page.set_default_timeout(60000)

        for page_num in range(1, 6):  # scrape first 5 pages
            print(f"🔍 Scraping page {page_num}...")
            url = f"{base_url}{page_num}"
            try:
                await page.goto(url, wait_until="networkidle", timeout=60000) # Set timeout for goto
            except PlaywrightTimeoutError: # Catch the specific timeout error
                print(f"⚠️ Timeout loading page {page_num}. Skipping to next page.")
                continue # Skip to the next page if timeout occurs
            except Exception as e:
                print(f"⚠️ Error loading page {page_num}: {e}. Stopping.")
                break # Stop for other errors


            product_elements = await page.query_selector_all("div.item-cell")
            if not product_elements:
                print(f"⚠️ No products found on page {page_num}. Stopping.")
                break

            for product in product_elements:
                try:
                    # Product name & link
                    title_el = await product.query_selector("a.item-title")
                    name = await title_el.inner_text() if title_el else None
                    link = await title_el.get_attribute("href") if title_el else None

                    # Specs (inside ul.item-features)
                    specs = {}
                    feature_items = await product.query_selector_all("ul.item-features li")
                    for li in feature_items:
                        text = await li.inner_text()
                        if ":" in text:
                            key, value = text.split(":", 1)
                            specs[key.strip()] = value.strip()

                    # Price
                    price_el = await product.query_selector("li.price-current")
                    price = None
                    if price_el:
                        price_text = await price_el.inner_text()
                        price = price_text.replace("\n", "").replace("–", "").strip()

                    all_products.append({
                        "Name": name,
                        "CAS Latency": specs.get("CAS Latency"),
                        "Voltage": specs.get("Voltage"),
                        "Multi-channel Kit": specs.get("Multi-channel Kit"),
                        "Timing": specs.get("Timing"),
                        "Model": specs.get("Model #"),
                        "Price": price,
                        "Link": link
                    })
                except Exception as e:
                    print(f"⚠️ Error extracting product: {e}")
                    continue
            await asyncio.sleep(2) # Add a small delay between pages

        await browser.close()

    df = pd.DataFrame(all_products)
    df.to_csv("newegg_ram.csv", index=False)
    print(f"✅ Done! {len(df)} RAM products saved to newegg_ram.csv")

# Run it
await scrape_newegg_ram()

🔍 Scraping page 1...
🔍 Scraping page 2...
⚠️ Timeout loading page 2. Skipping to next page.
🔍 Scraping page 3...
🔍 Scraping page 4...
⚠️ Timeout loading page 4. Skipping to next page.
🔍 Scraping page 5...
⚠️ Timeout loading page 5. Skipping to next page.
✅ Done! 72 RAM products saved to newegg_ram.csv


# **GPUs**

In [12]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import pandas as pd
import re
import time

# === Function to extract GPU specs from title text ===
def extract_gpu_specs(name):
    specs = {}
    if not name:
        return specs

    name = name.replace("-", " ").replace(",", " ")

    # GPU model (e.g., RTX 4070 Ti, RX 7800 XT)
    gpu_match = re.search(r"(RTX|RX|GTX)\s*\d{3,4}\s*(Ti|SUPER|XT|XTX)?", name, re.IGNORECASE)
    specs["GPU Model"] = gpu_match.group(0).upper() if gpu_match else None

    # Memory (e.g., 16GB, 12 GB GDDR6)
    mem_match = re.search(r"(\d{1,3})\s*GB", name, re.IGNORECASE)
    specs["Memory Size (GB)"] = mem_match.group(1) if mem_match else None

    # Memory type (GDDR6, GDDR6X, GDDR7)
    mem_type = re.search(r"(GDDR6X|GDDR6|GDDR7|GDDR5X|GDDR5)", name, re.IGNORECASE)
    specs["Memory Type"] = mem_type.group(1).upper() if mem_type else None

    # PCIe version
    pcie_match = re.search(r"PCI(?:\s*Express|\s*E)?\s*(\d+\.?\d*)", name, re.IGNORECASE)
    specs["PCIe Version"] = pcie_match.group(1) if pcie_match else None

    # Brand extraction (NVIDIA, AMD, Intel if appears)
    if re.search(r"RTX|GTX", name, re.IGNORECASE):
        specs["Chipset Brand"] = "NVIDIA"
    elif re.search(r"RX|RADEON", name, re.IGNORECASE):
        specs["Chipset Brand"] = "AMD"
    elif re.search(r"ARC", name, re.IGNORECASE):
        specs["Chipset Brand"] = "Intel"
    else:
        specs["Chipset Brand"] = None

    return specs


async def scrape_newegg_gpu():
    base_url = "https://www.newegg.com/p/pl?N=100006662&page="
    all_products = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Use a real browser header
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                          "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
        })

        page.set_default_timeout(60000)

        for page_num in range(1, 6):  # scrape first 5 pages
            print(f"🔍 Scraping page {page_num}...")
            url = f"{base_url}{page_num}"

            try:
                await page.goto(url, wait_until="domcontentloaded", timeout=60000)
                # Scroll to ensure all items load
                for i in range(5):
                    await page.mouse.wheel(0, 3000)
                    await asyncio.sleep(1)
            except PlaywrightTimeoutError:
                print(f"⚠️ Timeout on page {page_num}. Skipping.")
                continue

            product_elements = await page.query_selector_all("div.item-cell")
            if not product_elements:
                print(f"⚠️ No products found on page {page_num}. Stopping.")
                break

            for product in product_elements:
                try:
                    # Name & link
                    title_el = await product.query_selector("a.item-title")
                    name = await title_el.inner_text() if title_el else None
                    link = await title_el.get_attribute("href") if title_el else None

                    # Brand
                    brand_el = await product.query_selector("a.item-brand img")
                    brand = await brand_el.get_attribute("title") if brand_el else None

                    # Price
                    price_el = await product.query_selector("li.price-current")
                    price = await price_el.inner_text() if price_el else None
                    if price:
                        price = price.replace("\n", "").replace("–", "").strip()

                    # Shipping info
                    ship_el = await product.query_selector("div.product-delivery-title")
                    shipping = await ship_el.inner_text() if ship_el else None

                    # Extract specs from title
                    specs = extract_gpu_specs(name)

                    all_products.append({
                        "Name": name,
                        "Brand": brand,
                        "GPU Model": specs.get("GPU Model"),
                        "Chipset Brand": specs.get("Chipset Brand"),
                        "Memory Size (GB)": specs.get("Memory Size (GB)"),
                        "Memory Type": specs.get("Memory Type"),
                        "PCIe Version": specs.get("PCIe Version"),
                        "Price": price,
                        "Shipping": shipping,
                        "Link": link
                    })

                except Exception as e:
                    print(f"⚠️ Error reading GPU card: {e}")
                    continue

            await asyncio.sleep(2)

        await browser.close()

    df = pd.DataFrame(all_products)
    df.to_csv("newegg_gpu.csv", index=False)
    print(f"✅ Done! {len(df)} GPU products saved to newegg_gpu.csv")


# Run the scraper
await scrape_newegg_gpu()


🔍 Scraping page 1...
🔍 Scraping page 2...
⚠️ No products found on page 2. Stopping.
✅ Done! 36 GPU products saved to newegg_gpu.csv
